In [2]:
import sys
sys.path.append('..')

from app.database import _get_database_async
from rethinkdb import query as r

In [44]:
async def example_for_loop_async():
    async with await _get_database_async() as conn:
        sensor_table = r.table('sensors')
        async for sensor in await sensor_table.run(conn):
            print(sensor)


async def example_as_list_async():
    async with await _get_database_async() as conn:
        sensor_table = r.table('sensors')
        sensors = list((await sensor_table.run(conn)).items)
        print(sensors)

[{'coordinates': None, 'id': '4df86a94-dac5-499e-95b1-9ee7b8f65963', 'node_id': '!833c2233', 'region_id': '57b2b4c2-6d5a-4c4e-8d3b-5b4f4f4b4d54', 'uplink': False, 'watershed': None}]


In [22]:
from uuid import uuid4
from app.security import encode_jwt
# playground to test the async database and write rethinkdb queries
async def main():
    async with await _get_database_async() as conn:
        # match await r.table("sensors").get('4df86a94-dac5-499e-95b1-9ee7b8f65963').update(
        #     {"location": r.point(3, 2)},
        #     return_changes=True
        # ).run(conn):
        #     case {'changes': [{'new_val': sensor}]}:
        #         print(sensor)
        #     case _:
        #         print("No changes")

        # return await r.table("alarms").get('c5b2b4c2-6d5a-4c4e-8d3b-5b4f4f4b4d4d').update(
        #     {
        #         "region_id": "57b2b4c2-6d5a-4c4e-8d3b-5b4f4f4b4d54",
        #      },
        # ).run(conn)
        
        # await r.table("sensors").get('4df86a94-dac5-499e-95b1-9ee7b8f65963').update(
        #     {"location": None,
        #      "watershed": None
        #      },
        # ).run(conn)
        # return (await r.table("alarms").merge(lambda alarm: {
        #     "history": r.table("alarms_events").get_all(
        #         alarm["id"], index="alarm_id"
        #     ).merge(lambda event: {
        #         "records": r.table("alarms_event_records").get_all(event["id"], index="alarm_event_id").coerce_to("array")
        #     }).coerce_to("array")
        # }).run(conn)).items
        # async for region_id in await r.table("regions").get_field("id").run(conn):
        #     r.table('sensors').insert({"id": region_id, "channel_psk": str(uuid4())}, conflict='update').run(conn)
        # return await r.table("regions").get_field("id").run(conn)
        return await r.table('sensors').insert(
            [{"id": sensor["id"], "initialization_url": "http://localhost/init/" + encode_jwt({
            'id': sensor['id'],
            'node_id': sensor['node_id'],
        }) } async for sensor in await r.table("sensors").run(conn)],
            conflict='update'
        ).run(conn)
        # await r.table('sensors').replace(r.row.without('channel_psk')).run(conn)
    
await main()

SyntaxError: unterminated string literal (detected at line 37) (1582421117.py, line 37)